<h1>SCVx Astrobee (official_template)</h1>

$$
\begin{align*}
    \text{min} \quad &\int_{0}^{t_f}||\bm{\tau}(t)||_2^2dt+\int_0^{t_f}||\bm{a}(t)||_2^2dt\\
    \text{s.t.}\quad&
        \dot{\bm{r}}=
        \bm{v}\\
        &\dot{\bm{v}}=
        \frac{1}{m}\bm{F}=\bm{a}\\
        &\dot{\bm{q}}=
        \frac{1}{2}\begin{bmatrix}
        0&-\omega_x&-\omega_y&-\omega_z\\
        \omega_x&0&\omega_z&-\omega_y\\
        \omega_y&-\omega_z&0&\omega_x\\
        \omega_z&\omega_y&-\omega_x&0
        \end{bmatrix}\bm{q}\\
        &\dot{\bm{\omega}}=
        \bm{I}^{-1}(-\bm{\omega}\times\bm{I}\bm{\omega}+\bm{\tau})\\
    &||\bm{r}-\bm{c}_1||\geq d_1,\quad||\bm{r}-\bm{c}_2||\geq d_2,\quad||\bm{r}-\bm{c}_3||\geq d_3\\
    &||\bm{v}||\leq v_{\text{max}}\\
    &||\bm{\omega}||\leq \omega_{\text{max}}\\
    &||\bm{a}||_1\leq a_{\text{max}}\\
    &||\bm{\tau}||_1\leq\tau_{\text{max}}\\
    &\bm{x}(0)=\bm{x}_0,\quad\bm{x}(t_f)=\bm{x}_{t_f}\\
    &\bm{v}(0)=\bm{v}_0,\quad\bm{v}(t_f)=\bm{v}_{t_f}\\
    &\bm{q}(0)=\bm{q}_0,\quad\bm{q}(t_f)=\bm{q}_{t_f}\\
    &\bm{w}(0)=\bm{w}_0,\quad\bm{\omega}(t_f)=\bm{\omega}_{t_f}
\end{align*}
$$


Given the scaling matrices and vectors $\bm{S}_x$, $\bm{S}_u$, $\bm{c}_x$, $\bm{c}_u$:

$$\hat{\bm{u}}=\begin{bmatrix}
\bm{a}\\
\bm{\tau}
\end{bmatrix},\quad\hat{\bm{u}}=\bm{S}_u\bm{u}+\bm{c}_u$$

$$\hat{\bm{x}}=\begin{bmatrix}
\bm{r}\\
\bm{v}\\
\bm{q}\\
\bm{\omega}
\end{bmatrix},\quad\hat{\bm{x}}=\bm{S}_x\bm{x}+\bm{c}_x$$

Where scaling matrices and vectors try to limit the optimization variables within $\bm{x}\in[0, 1]^n$, $\bm{u}\in[0, 1]^m$.

Then, original optimization problem is transformed into (for instance, via 1st order Taylores series):

$$
\begin{align*}
    \text{min} \quad &J_\lambda(\bm{x},\bm{u},\bm{v}_c,\bm{v}_i)\\
    \text{s.t.}\quad&
        \dot{\bm{x}}=
        \bm{A}\bm{x}+\bm{B}\bm{u}+\bm{y}+\bm{v}_c\\
    &\bm{C}\bm{x}+\bm{D}\bm{u}+\bm{z}\leq \bm{v}_i\\
    &||\bm{x}-\bar{\bm{x}}||_\infty+||\bm{u}-\bar{\bm{u}}||_\infty\leq \eta\\
    &\bm{x}\in\mathcal{X},\quad \bm{u}\in\mathcal{U}
\end{align*}
$$

Where $\mathcal{X}$, $\mathcal{U}$ encompass convex restrictions.

Then, through discretization techniques, we need to generate:

$$
\begin{align*}
    \text{min} \quad &\mathcal{L}_\lambda(\bm{x},\bm{u},\bm{v}_c,\bm{v}_i)\\
    \text{s.t.}\quad&
        \bm{x}_{k+1}=
        \bm{A}_k\bm{x}+\bm{B}_k\bm{u}_k+\bm{y}_k+\bm{v}_{c_k}\\
    &\bm{C}_{k}\bm{x}_{k}+\bm{D}_{k}\bm{u}_{k}+\bm{z}_{k}\leq \bm{v}_{i_k}\\
    &||\bm{x}_{k}-\bar{\bm{x}}_{k}||_1+||\bm{u}_{k}-\bar{\bm{u}}_{k}||_1\leq \eta\\
    &\bm{x}_k\in\mathcal{X}_k,\quad \bm{u}_k\in\mathcal{U}_k
\end{align*}
$$

In [7]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import time
import ecos
import math

T = 51-1 # 101-1 means 101 discretization points
tf = 200.0
tau_double = tf/(T)
size_N=20
print("Step: ",tau," [s]")

start_r_xyz=np.array([[0.0],[-1.0],[1.2]])
end_r_xyz=np.array([[5.0],[2.0],[1.2]])

start_v_xyz=np.array([[0.0],[-1.0],[1.2]])
end_v_xyz=np.array([[5.0],[2.0],[1.2]])

start_quat=np.array([[0],[1],[0],[0]])
end_quat=np.array([[0],[0],[1],[0]])

start_w_xyz=np.array([[0.0],[0.0],[0.0]])
end_w_xyz=np.array([[0.0],[0.0],[0.0]])

start_pos_double=np.block([[start_r_xyz],[start_v_xyz],[start_quat],[start_w_xyz]])
end_pos_double=np.block([[end_r_xyz],[end_v_xyz],[end_quat],[end_w_xyz]])

c_obs1_double = np.array([[1.1],[-0.5],[1]]) #m
c_obs2_double = np.array([[2.6],[0.5],[1.1]]) #m
c_obs3_double = np.array([[4],[1.6],[1.2]]) #m

d_obs1_double = 0.8 #m
d_obs2_double = 0.8 #m
d_obs3_double = 0.8 #m

J1_double=0.1083 #Kg m^2
J2_double=0.1083 #Kg m^2
J3_double=0.1083 #Kg m^2

acc_max_double = 20/7.2*0.001 #m/s^2
torq_max_double = 100*0.000001 #Kg m^2/s^2

vel_max_double = 0.4 #m/s
omega_max_double = 5.0*np.pi/180 #rad/s


#Guidance parameters (COGU parameters)
rho0=0.0
rho1=0.1
rho2=0.7
etta0=0.001
etta1=10
beta_sh=2
beta_gr=2

lamb_double=1000
etta_double=1

e_tol=0.05
epsilon_stop_norm=0.04

#scaling matrices and vectors
S_u_scaling_double=np.array([[2*acc_max_double,0,0,0,0,0],
                      [0,2*acc_max_double,0,0,0,0],
                      [0,0,2*acc_max_double,0,0,0],
                      [0,0,0,2*torq_max_double,0,0],
                      [0,0,0,0,2*torq_max_double,0],
                      [0,0,0,0,0,2*torq_max_double],])
c_u_scaling_double=np.array([[-acc_max_double],
                      [-acc_max_double],
                      [-acc_max_double],
                      [-torq_max_double],
                      [-torq_max_double],
                      [-torq_max_double]])

S_x_scaling_double=np.array([[2*30,0,0,0,0,0,0,0,0,0,0,0,0],
                      [0,2*30,0,0,0,0,0,0,0,0,0,0,0],
                      [0,0,2*30,0,0,0,0,0,0,0,0,0,0],
                      [0,0,0,2*vel_max_double,0,0,0,0,0,0,0,0,0],
                      [0,0,0,0,2*vel_max_double,0,0,0,0,0,0,0,0],
                      [0,0,0,0,0,2*vel_max_double,0,0,0,0,0,0,0],
                      [0,0,0,0,0,0,2*1,0,0,0,0,0,0],
                      [0,0,0,0,0,0,0,2*1,0,0,0,0,0],
                      [0,0,0,0,0,0,0,0,2*1,0,0,0,0],
                      [0,0,0,0,0,0,0,0,0,2*1,0,0,0],
                      [0,0,0,0,0,0,0,0,0,0,omega_max_double,0,0],
                      [0,0,0,0,0,0,0,0,0,0,0,omega_max_double,0],
                      [0,0,0,0,0,0,0,0,0,0,0,0,omega_max_double]])
c_x_scaling_double=np.array([[-30],
                      [-30],
                      [-30],
                      [-vel_max_double],
                      [-vel_max_double],
                      [-vel_max_double],
                      [-1],
                      [-1],
                      [-1],
                      [-1],
                      [-omega_max_double],
                      [-omega_max_double],
                      [-omega_max_double]])

states_size = 13
inputs_size = 6
non_convex_inequalities_size = 3

Step:  tau  [s]


Cvxpy Optimization problem

In [57]:
x = cp.Variable((states_size, T + 1), name='x')
u = cp.Variable((inputs_size, T), name='u')
vc = cp.Variable((states_size, T), name='vc')
vi = cp.Variable((non_convex_inequalities_size, T+1), name='vi')

tau = cp.Parameter(name='tau')
sqrt_tau = cp.Parameter(name='sqrt_tau')

start_pos = cp.Parameter((states_size,1), name='start_pos')
end_pos = cp.Parameter((states_size,1), name='end_pos')

ox_cvxpy = cp.Parameter((states_size,T + 1), name='ox_cvxpy')
ou_cvxpy = cp.Parameter((inputs_size,T), name='ou_cvxpy')

S_x_scaling = cp.Parameter((states_size,states_size), name='S_x_scaling')
S_u_scaling = cp.Parameter((inputs_size,inputs_size), name='S_u_scaling')

c_x_scaling = cp.Parameter((states_size,1), name='c_x_scaling')
c_u_scaling = cp.Parameter((inputs_size,1), name='c_u_scaling')

A_discrete = cp.Parameter((states_size,states_size*T), name='A_discrete')
B_discrete = cp.Parameter((states_size,inputs_size*T), name='B_discrete')
y_discrete = cp.Parameter((states_size,1*T), name='y_discrete')

C_discrete = cp.Parameter((non_convex_inequalities_size,states_size*(T+1)), name='C_discrete')
D_discrete = cp.Parameter((non_convex_inequalities_size,inputs_size*(T+1)), name='D_discrete')
z_discrete = cp.Parameter((non_convex_inequalities_size,1*(T+1)), name='z_discrete')

lamb = cp.Parameter(name='lamb')
tau_lamb = cp.Parameter(name='tau_lamb')
etta = cp.Parameter(name='etta')

vel_max = cp.Parameter(name='vel_max')
omega_max = cp.Parameter(name='omega_max')

a_max = cp.Parameter(name='a_max')
torq_max = cp.Parameter(name='torq_max')

constraints = [
    x[:, 0] == start_pos[:,0],
    x[:, T] == end_pos[:,0]
]

cost = 0

for k in range(0, T):
    cost += cp.sum_squares(sqrt_tau*u[0:3,k:k+1])
    cost += cp.sum_squares(sqrt_tau*u[3:6,k:k+1])
    cost += cp.norm(tau_lamb*vc[0:states_size, k:k+1],1)

for k in range(0, T+1):
    cost += cp.norm(tau_lamb*vi[0:non_convex_inequalities_size, k:k+1],1)

for k in range(0, T): # from 0 to T-1
    constraints += [x[0:states_size, k+1:k+2] == A_discrete[0:states_size,states_size*k:states_size*(k+1)] @ x[0:states_size, k:k+1] + B_discrete[0:states_size,inputs_size*k:inputs_size*(k+1)] @ u[0:inputs_size, k:k+1]+y_discrete[0:states_size, k:k+1]+vc[0:states_size, k:k+1]]

for k in range(0, T): # from 0 to T-1
    constraints += [C_discrete[0:non_convex_inequalities_size,states_size*k:states_size*(k+1)] @ x[0:states_size, k:k+1] + D_discrete[0:non_convex_inequalities_size,inputs_size*k:inputs_size*(k+1)] @ u[0:inputs_size, k:k+1]+z_discrete[0:non_convex_inequalities_size, k:k+1]<=vi[0:non_convex_inequalities_size, k:k+1]]
    constraints  += [cp.norm(x[0:states_size, k:k+1]-ox_cvxpy[0:states_size,k],'inf')+cp.norm(u[0:inputs_size, k]-ou_cvxpy[0:inputs_size,k],'inf')<=etta]

for k in range(0, T+1):
    constraints += [cp.norm(S_x_scaling[3:6,3:6]@x[3:6,k:k+1]+c_x_scaling[3:6,0:1], 1)<=vel_max]
    constraints += [cp.norm(S_x_scaling[10:13,10:13]@x[10:13,k:k+1]+c_x_scaling[10:13,0:1], 1)<=omega_max]
for k in range(0, T):
    constraints += [cp.norm(S_u_scaling[0:3,0:3]@u[0:3,k:k+1]+c_u_scaling[0:3,0:1], 1)<=a_max]
    constraints += [cp.norm(S_u_scaling[3:6,3:6]@u[3:6,k:k+1]+c_u_scaling[3:6,0:1], 1)<=torq_max]

objective = cp.Minimize(cost)
problem = cp.Problem(objective, constraints)

In [58]:
print("Is DPP? ", problem.is_dcp(dpp=True))

Is DPP?  True
